# Optimization from scratch

In [ ]:
# Import symforce for the sole purpose of setting the value
# of epsilon, which must be done first and exactly once.
import symforce
symforce.set_epsilon_to_symbol()

import symforce.symbolic as sf
from symforce.values import Values
from symforce.opt.factor import Factor
from symforce.opt.optimizer import Optimizer
from symforce.opt.noise_models import PseudoHuberNoiseModel
from symforce.opt.noise_models import BarronNoiseModel
import sym

from pathlib import Path
import numpy as np
import secrets
import json

In [ ]:
with open('data.json', 'r') as f:
    data = json.load(f)

for k in data.keys():
    data[k] = np.array(data[k])

K = data['K']

In [ ]:
seed = secrets.randbits(32)
print(f'seeding RNG with {seed}')
rng = np.random.default_rng(seed)

## Triangulation (SymForce)

In [ ]:
def sf_projection(
    T_inC_ofW: sf.Pose3,
    p_inW: sf.V3,
    fx: sf.Scalar,
    fy: sf.Scalar,
    cx: sf.Scalar,
    cy: sf.Scalar,
    epsilon: sf.Scalar,
) -> sf.V2:
    """
    Symbolic function that projects a point into an image. (If the depth
    of this point is non-positive, then the projection will be pushed far
    away from the image center.)
    """
    p_inC = T_inC_ofW * p_inW
    z = sf.Max(p_inC[2], epsilon)   # <-- if depth is non-positive, then projection
                                    #     will be pushed far away from image center
    return sf.V2(
        fx * (p_inC[0] / z) + cx,
        fy * (p_inC[1] / z) + cy,
    )

def sf_projection_residual(
    T_inC_ofW: sf.Pose3,
    p_inW: sf.V3,
    q: sf.V2,
    fx: sf.Scalar,
    fy: sf.Scalar,
    cx: sf.Scalar,
    cy: sf.Scalar,
    epsilon: sf.Scalar,  
) -> sf.V2:
    """
    Symbolic function that computes the difference between a projected point
    and an image point.
    """
    q_proj = sf_projection(T_inC_ofW, p_inW, fx, fy, cx, cy, epsilon)
    return sf.V2(q_proj - q)

In [ ]:
# Create data structures
initial_values = Values(
    fx=K[0, 0],
    fy=K[1, 1],
    cx=K[0, 2],
    cy=K[1, 2],
    T_inB0_ofA=sym.Pose3(
        R=sym.Rot3.from_rotation_matrix(data['R_inB0_ofA']),
        t=data['p_inB0_ofA'],
    ),
    T_inB1_ofA=sym.Pose3(
        R=sym.Rot3.from_rotation_matrix(data['R_inB1_ofA']),
        t=data['p_inB1_ofA'],
    ),
    p_inA=data['p_inA'] + 0.5 * rng.standard_normal(size=3),
    b_0=data['b_0'],
    b_1=data['b_1'],
    epsilon=sym.epsilon,
)
optimized_keys = ['p_inA']
factors = [
    Factor(
        residual=sf_projection_residual,
        keys=[
            f'T_inB0_ofA',
            f'p_inA',
            f'b_0',
            'fx',
            'fy',
            'cx',
            'cy',
            'epsilon',
        ],
    ),
    Factor(
        residual=sf_projection_residual,
        keys=[
            f'T_inB1_ofA',
            f'p_inA',
            f'b_1',
            'fx',
            'fy',
            'cx',
            'cy',
            'epsilon',
        ],
    ),
]
   
# Create optimizer
optimizer = Optimizer(
    factors=factors,
    optimized_keys=optimized_keys,
    debug_stats=True,
    params=Optimizer.Params(
        iterations=100,
        use_diagonal_damping=True,
        lambda_down_factor=0.1,
        lambda_up_factor=5.,
        early_exit_min_reduction=1e-8,
    ),
)

In [ ]:
result = optimizer.optimize(initial_values)
assert(result.status == symforce.opt.optimizer.Optimizer.Status.SUCCESS)

## Triangulation (from scratch)

In [ ]:
mu = 0.
p_inA = initial_values['p_inA'].copy()
iter = 0

e_prev = get_e(p_inA)
while iter < 100:

    # Linearize
    # ...

    # Get the step u
    # ...

    # Get e_new and rel_reduction
    # ...
    
    # Show current status
    print(f'{iter:5d} : {e_prev:11.4e}, {e_new:11.4e} : {rel_reduction:11.4e}')
    
    # Take the step (update p_inA and set e_prev = e_new)
    # ...

    # Update iteration
    iter += 1

    # Stop if rel_reduction is small enough
    if (rel_reduction > 0) and (rel_reduction < 1e-10):
        break